In [1]:
import pandas as pd
from datetime import datetime, timedelta

def datemerge(ref_df, start_date, end_date):

        # Define start/ end date of the expected period:
    start_date = datetime.strptime(start_date, '%Y-%m-%d')
    end_date = datetime.strptime(end_date, '%Y-%m-%d')

        # Create the Dataframe with index is the list of date from expected period:
    df = pd.DataFrame(index=[start_date + timedelta(days=day) for day in range((end_date - start_date).days + 1)])
    df.index.name = 'PublishedDate'

    output = pd.merge(df, ref_df, on='PublishedDate', how='left').fillna(value=0)

    return output

In [2]:
def hyperlink(df, original_df, topORcom):
    # Take the top 10 most frequent SiteName/Author:
    df = pd.DataFrame(df).sort_values(by='Id', ascending=False).head(10)
    # Necessary variables declaration:
    out_put = pd.DataFrame()
    if topORcom == 'top':
        topic_or_comment = 'UrlTopic'
    elif topORcom == 'com':
        topic_or_comment = 'UrlComment'
    else:
        print('Wrong input of topORcom')
    # Create the output Dataframe:
    for each_record in list(df.index):
        link_df = original_df[original_df[df.index.name] == each_record].pivot_table(index=topic_or_comment, values='Id', aggfunc='count').sort_values(by='Id', ascending=False)
        out_put.loc[each_record, 'Link'] = ('=' + 'HYPERLINK(' + '"' + str(list(link_df.index)[0]) + '",' + '"' + str(each_record) + '")')
    out_put.index.name = df.index.name

    out_put = pd.merge(out_put, df, on=df.index.name, how='left').set_index('Link')

    return out_put


In [22]:
def to_excel_file(file_name, dict_sheet_df):

    file_directory = "D" + ":" + "\ ".strip() + "Data Kompa" + "\ ".strip() + "Hotpot" + "\ ".strip() + str(file_name) + ".xlsx"
    with pd.ExcelWriter(file_directory) as writer:
        for slide, dataframe in dict_sheet_df.items():

            if type(dataframe) == list:
                start_column = 0
                for each_df in dataframe:
                    each_df.to_excel(writer, sheet_name=slide, startcol=start_column)
                    start_column += len(each_df.axes[1]) + 3

            if (type(dataframe) == pd.DataFrame) or (type(dataframe) == pd.Series):
                dataframe.to_excel(writer, sheet_name=slide)

In [4]:
import pandas as pd
import vaex as vx
import numpy as np
from datemerge import datemerge
from hyperlink import hyperlink
from to_excel_file import to_excel_file

In [5]:
df = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Data')

In [6]:
sentiment = ['Positive','Neutral','Negative']
main = ['Ashima','KTOP','Mala Gang']
compe = ['Asanoha','Momo Paradise','Dookki','Mala Panda','Yangguofu']
all_brands = main + compe
channel = ['Facebook','Fanpage','Food Review','Forum','News','Tiktok','Youtube','Google Review','Instagram','Threads','Linkedin','E-commerce']

In [7]:
Link = r"D:\\Data Kompa\\Hotpot\\OutputQ1.xlsx"

In [8]:
#SHEET sentiment_interaction

#sentiment
df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']] = df[['Sentiment','Labels1','Labels2','Likes','Shares','Comments']].fillna(0)

SOV = df.pivot_table(index='Topic',columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
SOV.columns = [col[1] if isinstance(col, tuple) else col for col in SOV.columns]
SOV.rename(columns={0: 'Khác'},inplace=True)
SOV['Total'] = SOV[['Positive', 'Neutral', 'Negative', 'Khác']].sum(axis=1)
SOV = SOV[['Topic', 'Positive', 'Neutral', 'Negative', 'Khác', 'Total']]
df['Topic'] = pd.Categorical(df['Topic'], categories=all_brands, ordered=True)
df = df.sort_values('Topic').reset_index(drop=True)
print(SOV)

#Interaction
df['Topic'] = df['Topic'].str.strip()
df_main = df[df['Topic'].isin(main)].reset_index()
inter = df_main.pivot_table(index='Topic', values=['Comments', 'Shares', 'Likes'], aggfunc='sum').fillna(0).reset_index()
inter = inter[['Topic','Comments','Shares','Likes']]


count_post = df_main.pivot_table(index='Topic',columns='post',values='Id',aggfunc='count').fillna(0).reset_index()
count_post

author = df_main.drop_duplicates(subset=['Topic', 'AuthorId']).pivot_table(index='Topic',values='AuthorId',aggfunc='count').reset_index()
author

interaction = pd.merge(pd.merge(count_post, inter, on='Topic'), author, on='Topic')
print('\n',interaction)

           Topic  Positive  Neutral  Negative     Khác     Total
0        Asanoha     977.0    274.0       9.0    681.0    1941.0
1         Ashima     209.0   3317.0      13.0   1259.0    4798.0
2         Dookki   12629.0  63088.0    1626.0  46867.0  124210.0
3           KTOP      18.0    666.0      12.0      0.0     696.0
4      Mala Gang     274.0   2528.0      30.0    397.0    3229.0
5     Mala Panda     148.0    200.0      13.0     27.0     388.0
6  Momo Paradise       0.0     12.0       0.0      0.0      12.0
7      Yangguofu      37.0     28.0       0.0      0.0      65.0

        Topic  post MXH  post News  Comments  Shares    Likes  AuthorId
0     Ashima      3789        116    5537.0   804.0  60364.0       950
1       KTOP       637         14    1621.0   153.0   1765.0        98
2  Mala Gang      2074         14    2880.0   386.0  49397.0       892


In [9]:
with pd.ExcelWriter(Link, engine='openpyxl') as writer:
    SOV.to_excel(writer, sheet_name='Sentiment_Interaction', startrow=0, startcol=0, index=False)
    interaction.to_excel(writer, sheet_name='Sentiment_Interaction', startrow=0, startcol=9, index=False)

In [10]:
#SHEET sentiment.channel
sent_chn = df.pivot_table(index=['Topic','Sentiment','POE'],columns=['Channel'],values='Id',aggfunc='count').reset_index().reindex(columns = ['Topic','Sentiment','POE'] + channel).fillna(0)
sent_chn = sent_chn[sent_chn['Sentiment'] != 0]

total_poe = df.pivot_table(index=['Topic','Sentiment','POE'],values='Id',aggfunc='count').reset_index()
total_setiment_topic = df.pivot_table(index=['Topic','Sentiment'],values='Id',aggfunc='count').reset_index()
total_poe = pd.merge(total_poe,total_setiment_topic,on=['Topic','Sentiment'])
total_poe['per_POE_by_sent'] = total_poe['Id_x']/total_poe['Id_y'] 
total_poe = total_poe.drop(columns=['Id_x','Id_y'])

sent_chn = pd.merge(sent_chn,total_poe,on=['Topic','Sentiment','POE'])

sent_chn

,Topic,Sentiment,POE,Facebook,Fanpage,Food Review,Forum,News,Tiktok,Youtube,Google Review,Instagram,Threads,Linkedin,E-commerce,per_POE_by_sent
0,Asanoha,Negative,Earned,0.0,0.0,0.0,0.0,0.0,8.0,0.0,1.0,0.0,0.0,0.0,0.0,1.000000
1,Asanoha,Neutral,Earned,36.0,0.0,0.0,0.0,0.0,113.0,0.0,0.0,1.0,0.0,1.0,0.0,0.551095
2,Asanoha,Neutral,Owned,0.0,110.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0,0.0,0.0,0.0,0.448905
3,Asanoha,Positive,Earned,0.0,0.0,0.0,0.0,0.0,967.0,0.0,5.0,0.0,0.0,0.0,0.0,0.994882
4,Asanoha,Positive,Owned,0.0,5.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.005118
5,Ashima,Negative,Earned,1.0,0.0,0.0,0.0,7.0,2.0,0.0,3.0,0.0,0.0,0.0,0.0,1.000000
6,Ashima,Neutral,Earned,2611.0,0.0,59.0,1.0,72.0,12.0,1.0,0.0,10.0,0.0,0.0,0.0,0.833886
7,Ashima,Neutral,Owned,0.0,430.0,0.0,0.0,0.0,58.0,0.0,0.0,33.0,0.0,0.0,0.0,0.157070
8,Ashima,Neutral,Paid,12.0,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0,0.0,0.009044
9,Ashima,Positive,Earned,79.0,0.0,0.0,0.0,0.0,6.0,0.0,30.0,0.0,0.0,0.0,0.0,0.550239


In [11]:
#SHEET sentiment_labels
Labels = df.pivot_table(index=['Topic','Labels1','Labels2'],columns='Sentiment',values='Id',aggfunc='count').fillna(0).reset_index()
Labels = Labels[Labels['Labels1'] != 0]
Labels = Labels[['Topic','Labels1','Labels2','Positive','Neutral','Negative']]
Labels

Sentiment,Topic,Labels1,Labels2,Positive,Neutral,Negative
1,Asanoha,Chương trình MKT,Bài đăng quảng bá,111.0,135.0,0.0
2,Asanoha,Chương trình MKT,Món ăn mới / Thức uống mới,0.0,4.0,0.0
3,Asanoha,Cơ sở vật chất,Hình ảnh không gian,10.0,0.0,1.0
4,Asanoha,Cơ sở vật chất,Tình trạng vệ sinh,3.0,0.0,0.0
5,Asanoha,Dịch vụ,Phục vụ,4.0,0.0,1.0
...,...,...,...,...,...,...
111,Yangguofu,Chương trình MKT,Bài đăng quảng bá,16.0,26.0,0.0
112,Yangguofu,Thương hiệu,So sánh thương hiệu,1.0,0.0,0.0
113,Yangguofu,Thương hiệu,Sẵn lòng trải nghiệm,5.0,0.0,0.0
114,Yangguofu,Thương hiệu,Đề cập đến thương hiệu,2.0,2.0,0.0


In [12]:
with pd.ExcelWriter(Link, engine='openpyxl', mode='a') as writer:
    sent_chn.to_excel(writer, sheet_name='Sentiment.Channel', startrow=0, startcol=0, index=False)
    Labels.to_excel(writer, sheet_name='Sentiment.Labels', startrow=0, startcol=0, index=False)

REPORT

mainbrand

In [13]:
period = 'Quý'
period_time = "Quý 1 2025"

In [14]:
sent = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Interaction')
chn = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Channel')
lb = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\Tính Hotpot.xlsx", sheet_name='Sentiment.Labels')
#demo = pd.read_excel(r"D:\\Data Kompa\\Hotpot\\HOTPOTquar2.xlsx", sheet_name='Demo')

In [15]:
slide_n_dict = {}

In [16]:
#chn_main = chn[(chn['Quý'] == quar)]

df_list=[]
for i in main:
    df_chn = chn[(chn[period] == period_time) & (chn['Brand']==i)]
    overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
    df_list.append(overall)
    
    
    total_overall = df_chn[['MXH', 'News', 'Total final']].sum()
    sent_chn_o =(
    df_chn.groupby(['Sentiment'])[['MXH', 'News', 'Total final']]
    .sum()
    .div(total_overall)
    .fillna(0)
    .round(3)
    .reindex(sentiment)
    .reset_index())
    df_list.append(sent_chn_o)
    
    
    df_working = df[df['Topic']==i]
    date = datemerge(df_working.groupby(['PublishedDate']).agg({'Id': 'count'}).pivot_table(index='PublishedDate', values='Id', aggfunc='sum'), '2025-01-01', '2025-03-31')
    df_list.append(date)
    
    sent_total = df_chn.groupby(['Sentiment'])['Total final'].sum().round(0).reindex(sentiment).reset_index()
    df_list.append(sent_total)
    
    df_lb = lb[(lb[period] == period_time) & (lb['Brand']==i)]
    labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
    df_list.append(labels)
    

    sent_by_chn = df_chn.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
    sent_by_chn = sent_by_chn.transpose().reindex(channel, columns = sentiment)
    sent_by_chn['Total'] = sent_by_chn.sum(axis=1)
    df_list.append(sent_by_chn)
    
    poe = df_chn.pivot_table(index='Brand',columns='Note',values='Total final',aggfunc=sum).round(0)
    poe['Total'] = poe.sum(axis=1).round(0)
    df_list.append(poe)
    
    for j in ['Paid','Owned','Earned']:
        df_chn_poe = df_chn[df_chn['Note']==j]
        sent_by_chn_poe = df_chn_poe.pivot_table(index=['Sentiment'], values=channel, aggfunc='sum').round(0)
        sent_by_chn_poe['Total_sent'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe = sent_by_chn_poe.transpose().reindex(channel, columns = sentiment).fillna(0)
        sent_by_chn_poe['Total'] = sent_by_chn_poe.sum(axis=1)
        sent_by_chn_poe.index.name = str(i) + '_' + str(j)
        df_list.append(sent_by_chn_poe)
    

    lb_chn = df_working.pivot_table(index='Labels2', columns='Channel', values='Id', aggfunc='count').reindex(columns=channel).fillna(0)
    # Calculate row sums for sorting
    row_sums = lb_chn.sum(axis=1)
    lb_chn = lb_chn.loc[row_sums.sort_values(ascending=False).index]
    # Normalize each row by its total sum and round to 3 decimal places
    lb_chn = lb_chn.div(lb_chn.sum(axis=1), axis=0).round(3).reset_index()
    lb_chn = lb_chn.drop(lb_chn[lb_chn["Labels2"] == 0].index).head(10)
    df_list.append(lb_chn)

    for k in ['Positive','Negative']:
        labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
        df_list.append(labels_sent)
        
    for i_sent in ['Positive', 'Negative']:
        df_sentiment = df_working[df_working['Sentiment'] == i_sent]
        top_sources = (
            df_sentiment.groupby(['Title', 'Channel', 'UrlTopic'])
            .agg({'Id': 'count'})
            .sort_values(by='Id', ascending=False)
            .head(10)
            .reset_index())
        top_sources['Link'] = top_sources.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:30] + ' ...")', axis=1)
        top_sources = top_sources.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'Channel', 'Id'], axis=1).set_index('Link')
        top_sources.index.name = f"Top Sources ({i_sent})"
        df_list.append(top_sources)
        
    top_sources_cmt = (
    df_working.groupby(['Title', 'SiteName', 'UrlTopic'])
        .agg({'Comments': 'sum'})
        .sort_values(by='Comments', ascending=False)
        .head(10)
        .reset_index())
    top_sources_cmt['Link'] = top_sources_cmt.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:50] + ' ...")', axis=1)
    top_sources_cmt = top_sources_cmt.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'SiteName', 'Comments'], axis=1).set_index('Link')
    top_sources_cmt.index.name = f"Top Comments ({i})"
    df_list.append(top_sources_cmt)
    
    top_sources_shares = (
    df_working.groupby(['Title', 'SiteName', 'UrlTopic'])
        .agg({'Shares': 'sum'})
        .sort_values(by='Shares', ascending=False)
        .head(10)
        .reset_index())
    top_sources_shares['Link'] = top_sources_shares.apply(lambda row: '=HYPERLINK("' + row['UrlTopic'] + '","' + row['Title'][:50] + ' ...")', axis=1)
    top_sources_shares = top_sources_shares.drop(['Title', 'UrlTopic'], axis=1).reindex(['Link', 'SiteName', 'Shares'], axis=1).set_index('Link')
    top_sources_shares.index.name = f"Top Shares ({i})"
    df_list.append(top_sources_shares)

    
#print(df_list)  

C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_11840\2002143966.py:6: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  overall = df_chn.groupby('Brand')['MXH', 'News', 'Total final'].sum().reset_index()
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_11840\2002143966.py:30: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels = df_lb.groupby(['Labels2'])['Brand','Labels2','Total final'].sum().round(0).sort_values(by='Total final',ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_11840\2002143966.py:63: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  labels_sent = df_lb.groupby('Labels2')['Labels2',k].sum().round(0).sort_values(by=k ,ascending = False)
C:\Users\Kompa-KTelt056\AppData\Local\Temp\ipykernel_11840\2002143966

In [17]:
Ashima = []
KTOP = []
Mala_Gang = []

counts = len(df_list)
df_counts = 0  # Counter for the DataFrames

for df_item in df_list:
    df_counts += 1  # Increment the counter
    
    if df_counts < (counts/3+1):
        Ashima.append(df_item)
        slide_n_dict['Ashima'] = Ashima
    
    elif df_counts < ((counts/3)*2+1):
        KTOP.append(df_item)
        slide_n_dict['KTOP'] = KTOP
        
    else:
        Mala_Gang.append(df_item)
        slide_n_dict['Mala Gang'] = Mala_Gang
        
#print(slide_n_dict)

competitors

In [18]:
topic_groups = {
    'Ashima & compe': ['Ashima', 'Asanoha', 'Momo Paradise'],
    'KTOP & compe': ['KTOP', 'Dookki'],
    'Malagang & compe': ['Mala Gang', 'Mala Panda', 'Yangguofu', 'Dookki']
}

In [19]:

for group_name, topics in topic_groups.items():
    
    df_compe_list =[]
    
    sent_compe = sent[(sent[period] == period_time) & sent['Brand'].isin(topics)].copy()
    sent_compe = sent_compe.groupby('Brand')[['Total final','Positive','Neutral','Negative']].sum().reset_index()
    sent_compe.loc[:, 'NSR'] = (sent_compe['Positive']-sent_compe['Negative'])/(sent_compe['Positive']+sent_compe['Negative'])
    NSR = sent_compe[['Total final','NSR','Brand']].round(3)
    df_compe_list.append(NSR)
    df_compe_list.append(sent_compe)
    
    lb_sent_compe = lb[(lb[period] == period_time) & lb['Brand'].isin(topics)]
    total_brand = lb_sent_compe.groupby('Brand')[['Positive','Neutral','Negative']].sum().sum(axis=1)
    total_brand.name = 'total_brand'
    labels = lb_sent_compe.groupby(['Brand','Labels2'])[['Positive','Neutral','Negative']].sum().fillna(0).round(3).reset_index()
    labels = pd.merge(labels, total_brand, how='left', on='Brand')
    labels[['Positive', 'Neutral', 'Negative']] = labels[['Positive', 'Neutral', 'Negative']].div(labels['total_brand'], axis=0).round(3)
    labels['total_brand'] = labels[['Positive', 'Neutral', 'Negative']].sum(axis=1)
    labels = labels.sort_values(by= ['Brand','total_brand'], ascending = [True,False])
    for brand in topics:
        labels_brand = labels[labels['Brand'] == brand].head(5)
        labels_brand.index.name = 'Sent_Lb'
        df_compe_list.append(labels_brand)
    
    lb_per = lb_sent_compe.groupby(['Brand','Labels2'])['Total final'].sum().fillna(0).round(3).reset_index()
    total_per_brand = lb_sent_compe.groupby('Brand')['Total final'].sum().fillna(0).round(3)
    total_per_brand.name = 'total_per_brand'
    lb_per = pd.merge(lb_per, total_per_brand, how='left', on='Brand')
    lb_per['percentage'] = (lb_per['Total final'] / lb_per['total_per_brand'])
    lb_per = lb_per[['Brand','Labels2','percentage']].sort_values(by = ['Brand','percentage'], ascending = [True,False])
    
    for brand in topics:
        lb_per_brand = lb_per[lb_per['Brand'] == brand].head(10)
        lb_per_brand.index.name = '%Labels'
        df_compe_list.append(lb_per_brand)
    
    
    df_working_compe = df[df['Topic'].isin(topics)]
    date_compe = datemerge(df_working_compe.groupby(['PublishedDate','Topic']).agg({'Id': 'count'}).pivot_table(index=['PublishedDate'], values='Id', columns='Topic',aggfunc='sum'), '2025-01-01', '2025-03-31')
    df_compe_list.append(date_compe)
    
    df_fp = df[(df['Topic'].isin(topics)) & (df['Channel']=='Fanpage') & (df['Type']=='fbPageTopic')]
    fp_inter = df_fp.pivot_table(index='Topic',values=['Comments','Shares','Likes'], aggfunc = sum)
    fp_inter['post'] = df_fp.groupby('Topic')['Id'].count()
    df_compe_list.append(fp_inter)
    
    slide_n_dict[group_name] = df_compe_list

In [23]:
to_excel_file('HotpotOutput', slide_n_dict)